In [1]:
import numpy as np
import pandas as pd

In [2]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-11-24 22:56:04.305553: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-24 22:56:04.484950: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 22:56:05.291975: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/lib/x86_64-linux-gnu/:/opt/conda/lib
2024-11-24 22:56:05.292074: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] 

In [3]:
# Define paths
root_dir = './Data Files'    # CHANGE BASED ON FOLDER LOCATION
sub_folder = 'short axis frames'
dataframe_path = './Data Files/new_image_labels_gcp.csv'  # Update the path

In [4]:
 # Load DataFrames
original_data = pd.DataFrame()
for video_folder in os.listdir(os.path.join(root_dir, sub_folder)):
    video_path = os.path.join(root_dir, sub_folder, video_folder)
    if os.path.isdir(video_path):
        label_file = os.path.join(root_dir, 'shortaxis_binary v2.xlsx')
        try:
            labels_df = pd.read_excel(label_file, sheet_name=video_folder)
        except ValueError:
            continue
        
        for img_filename in os.listdir(video_path):
            if img_filename.endswith(".jpg"):
                img_path = os.path.join(video_path, img_filename)
                frame_idx = int(os.path.splitext(img_filename)[0].split('_')[-1])
                labels = labels_df.loc[frame_idx, ['BAD QUALITY', 'CORD', 'FLUID']].values.astype('float32')
                new_data = {
                    'FILENAME': img_path,
                    'BAD QUALITY': labels[0],
                    'CORD': labels[1],
                    'FLUID': labels[2]}
                original_data = pd.concat([original_data, pd.DataFrame(new_data, index=[0])], axis=0)

new_data = pd.read_csv(dataframe_path)

# Combine datasets
combined_data = pd.concat([original_data, new_data], ignore_index=True)

# Split datasets
train_data, temp_data = train_test_split(combined_data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.25, random_state=42)

# Define ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1.0 / 255, rotation_range=15, zoom_range=0.2, horizontal_flip=True)
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Helper function to create generators
def create_generator(datagen, dataframe, batch_size, target_size=(128, 128)):
    return datagen.flow_from_dataframe(
        dataframe,
        x_col='FILENAME',
        y_col=['BAD QUALITY', 'CORD', 'FLUID'],
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        shuffle=False,
        validate_filenames = False
    )

# Create generators
batch_size = 8
train_generator = create_generator(train_datagen, train_data, batch_size)
val_generator = create_generator(val_test_datagen, val_data, batch_size)
test_generator = create_generator(val_test_datagen, test_data, batch_size)

Found 3256 non-validated image filenames.
Found 611 non-validated image filenames.
Found 204 non-validated image filenames.


In [5]:
from keras.layers import ConvLSTM2D, Dropout, Flatten, Dense
from keras.models import Sequential

In [6]:
import numpy as np

def create_sequence_generator(data_generator, time_steps, batch_size):
    """
    Transforms a data generator to output sequences for LSTM or TimeDistributed models.
    
    Args:
    - data_generator: Original data generator (e.g., train_generator).
    - time_steps: Number of frames in each sequence.
    - batch_size: Number of sequences per batch.
    
    Returns:
    - Generator yielding batches of sequences (batch_size, timesteps, height, width, channels) and labels.
    """
    # Buffer to store frames and labels for creating sequences
    frame_buffer = []
    label_buffer = []

    for batch_x, batch_y in data_generator:
        # Append frames and labels to the buffers
        for i in range(len(batch_x)):
            frame_buffer.append(batch_x[i])
            label_buffer.append(batch_y[i])

            # When we have enough frames to form a sequence
            if len(frame_buffer) >= time_steps:
                # Form a sequence
                sequence_x = np.array(frame_buffer[:time_steps])  # First `time_steps` frames
                sequence_y = np.array(label_buffer[time_steps - 1])  # Label of the last frame

                # Remove used frames and labels from the buffer
                frame_buffer.pop(0)
                label_buffer.pop(0)

                # Yield a batch of sequences
                yield (
                    np.expand_dims(sequence_x, axis=0),  # Add batch dimension
                    np.expand_dims(sequence_y, axis=0)   # Add batch dimension
                )

In [7]:
time_steps = 10  # Define the number of frames per sequence

# Create sequence generators for train and validation
train_sequence_generator = create_sequence_generator(train_generator, time_steps, 16)
val_sequence_generator = create_sequence_generator(val_generator, time_steps, 16)

In [8]:
from keras.applications import VGG16, ResNet50, ResNet152
from keras.layers import (
    Input, Dense, Dropout, LSTM, TimeDistributed, GlobalAveragePooling2D, BatchNormalization
)
from keras.models import Model

# Parameters
image_height, image_width, no_of_channels = 128, 128, 3
timesteps = 10
no_of_classes = 3  # BAD QUALITY, CORD, FLUID

# Load pre-trained VGG16
ResNet_base = ResNet152(input_shape=(image_height, image_width, no_of_channels), 
                   weights="imagenet", include_top=False)

# Add global average pooling to get fixed-length feature vectors
ResNet_output = GlobalAveragePooling2D()(ResNet_base.output)
ResNet_model = Model(ResNet_base.input, ResNet_output)
ResNet_model.trainable = True  # Freeze the VGG16 base

# Input for video sequences
video_input = Input(shape=(timesteps, image_height, image_width, no_of_channels))

# Encode each frame using the VGG16 model (shared across frames)
video_frames_encoded = TimeDistributed(ResNet_model)(video_input)

# Process the encoded sequence using LSTM
video_frames_encoded_sequence = LSTM(1024, return_sequences=False)(video_frames_encoded)
video_frames_encoded_sequence = Dropout(0.25)(video_frames_encoded_sequence)

# Fully connected layers with Dropout and Batch Normalization
hidden_layer1 = Dense(1024, activation="relu")(video_frames_encoded_sequence)
hidden_layer1 = BatchNormalization()(hidden_layer1)
hidden_layer1 = Dropout(0.2)(hidden_layer1)

hidden_layer2 = Dense(512, activation="relu")(hidden_layer1)
hidden_layer2 = BatchNormalization()(hidden_layer2)
hidden_layer2 = Dropout(0.2)(hidden_layer2)

hidden_layer3 = Dense(256, activation="relu")(hidden_layer2)
hidden_layer3 = BatchNormalization()(hidden_layer3)
hidden_layer3 = Dropout(0.2)(hidden_layer3)

hidden_layer4 = Dense(128, activation="relu")(hidden_layer3)
hidden_layer4 = BatchNormalization()(hidden_layer4)
hidden_layer4 = Dropout(0.2)(hidden_layer4)

hidden_layer5 = Dense(64, activation="relu")(hidden_layer4)
hidden_layer5 = BatchNormalization()(hidden_layer5)
hidden_layer5 = Dropout(0.2)(hidden_layer5)

# Output layer for multi-label classification
outputs = Dense(no_of_classes, activation="sigmoid")(hidden_layer5)

# Define the model
model = Model([video_input], outputs)

2024-11-24 22:58:14.974626: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-24 22:58:15.013358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-24 22:58:15.016898: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-24 22:58:15.020580: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

234698864/234698864 [==============================] - 8s 0us/step


In [9]:
# Compile the model
model.compile(
    optimizer="adam", 
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)

In [10]:
# Model Summary
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10, 128, 128, 3)  0         
                             ]                                   
                                                                 
 time_distributed (TimeDistr  (None, 10, 2048)         58370944  
 ibuted)                                                         
                                                                 
 lstm (LSTM)                 (None, 1024)              12587008  
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 1024)              1049600   
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096

In [11]:
# Train the model
model.fit(
    train_sequence_generator,
    validation_data=val_sequence_generator,
    epochs=10,
    steps_per_epoch=len(train_generator) // time_steps,
    validation_steps=len(val_generator) // time_steps
)

Epoch 1/10


2024-11-24 22:59:09.999280: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8900


40/40 [==============================] - 41s 302ms/step - loss: 0.6671 - accuracy: 0.6000 - val_loss: 0.6321 - val_accuracy: 0.4286
Epoch 2/10
40/40 [==============================] - 7s 174ms/step - loss: 0.5878 - accuracy: 0.6500 - val_loss: 0.4800 - val_accuracy: 0.7143
Epoch 3/10
40/40 [==============================] - 7s 175ms/step - loss: 0.5369 - accuracy: 0.5750 - val_loss: 0.7612 - val_accuracy: 0.1429
Epoch 4/10
40/40 [==============================] - 7s 175ms/step - loss: 0.5193 - accuracy: 0.5250 - val_loss: 1.6624 - val_accuracy: 0.1429
Epoch 5/10
40/40 [==============================] - 7s 176ms/step - loss: 0.4540 - accuracy: 0.6500 - val_loss: 5.5950 - val_accuracy: 0.5714
Epoch 6/10
40/40 [==============================] - 7s 177ms/step - loss: 0.3880 - accuracy: 0.7250 - val_loss: 22.7617 - val_accuracy: 0.4286
Epoch 7/10
40/40 [==============================] - 7s 178ms/step - loss: 0.5154 - accuracy: 0.5000 - val_loss: 23.1754 - val_accuracy: 0.5714
Epoch 8/10
40/